In [1]:
import random
import pandas as pd
import os

data_path = r"data"
imaging_path = os.path.join(data_path, "imaging")

filenames = os.listdir(imaging_path)

In [4]:
# get random subset with even amount of slobe and llobe
random.seed(42)
slobe_files = [f for f in filenames if 'slobe' in f.lower()]
llobe_files = [f for f in filenames if 'llobe' in f.lower()]
random_slobe = random.sample(slobe_files, 10)
random_llobe = random.sample(llobe_files, 10)
random_subset = random_slobe + random_llobe
# copy images over to imaging_subset folder
if input("copy images to imaging_subset folder? (y/n): ") == 'y':
    subset_path = os.path.join(data_path, "imaging_subset")
    os.makedirs(subset_path, exist_ok=True)
    # make sure no existing files exist
    if os.listdir(subset_path):
        raise Exception("imaging_subset folder is not empty")
    for f in random_subset:
        src = os.path.join(imaging_path, f)
        dst = os.path.join(subset_path, f)
        if not os.path.exists(dst):
            os.system(f'cp "{src}" "{dst}"')
            print(f"Copied {f} to imaging_subset folder.")
        else:
            print(f"{f} already exists in imaging_subset folder.")
else:
    print("images not copied.")

Copied MASK_SK658 Slobe ch010119.jpg to imaging_subset folder.
Copied MASK_SK658 Slobe ch010077.jpg to imaging_subset folder.
Copied MASK_SK658 Slobe ch010115.jpg to imaging_subset folder.
Copied MASK_SK658 Slobe ch010111.jpg to imaging_subset folder.
Copied MASK_SK658 Slobe ch010104.jpg to imaging_subset folder.
Copied MASK_SK658 Slobe ch010066.jpg to imaging_subset folder.
Copied MASK_SK658 Slobe ch010063.jpg to imaging_subset folder.
Copied MASK_SK658 Slobe ch010103.jpg to imaging_subset folder.
Copied MASK_SK658 Slobe ch010135.jpg to imaging_subset folder.
Copied MASK_SK658 Slobe ch010089.jpg to imaging_subset folder.
Copied MASK_Sk658 Llobe ch010030.jpg to imaging_subset folder.
Copied MASK_Sk658 Llobe ch010071.jpg to imaging_subset folder.
Copied MASK_Sk658 Llobe ch010061.jpg to imaging_subset folder.
Copied MASK_Sk658 Llobe ch010051.jpg to imaging_subset folder.
Copied MASK_Sk658 Llobe ch010174.jpg to imaging_subset folder.
Copied MASK_Sk658 Llobe ch010017.jpg to imaging_subset 